In [ ]:
from collections import namedtuple
import onekey_algo.custom.components as okcomp
from onekey_algo import OnekeyDS as okds

import pandas as pd
import numpy as np

# 读取数据，B超诊断阳性=1，bc_data.csv是要读取的数据。
import pandas as pd
import os
os.makedirs('img', exist_ok=True)
os.makedirs('features', exist_ok=True)

# 读取数据，B超诊断阳性=1，bc_data.csv是要读取的数据。
features = pd.read_csv('features/clinical_sel.csv')
features['ID'] = features['ID'].map(lambda x: x.replace('.nii.gz', ''))
labels = ['BCR', 'BCR_TIME']
featrues_not_use = ['ID', 'HRR_ANY']

label_data = pd.read_csv('data/CPGEA-TCGA 20230106 OK.csv', header=0)[['ID', labels[0], labels[1]]]
label_data['ID'] = label_data['ID'].astype(str)
structed_data = pd.merge(features, label_data, left_on='ID', right_on='ID', how='inner')
structed_data

In [ ]:
from collections import namedtuple
import onekey_algo.custom.components as okcomp
from onekey_algo import OnekeyDS as okds
from onekey_algo.custom.utils import map2numerical
from onekey_algo.custom.components.comp1 import fillna
import onekey_algo.custom.components as okcomp
from lifelines import CoxPHFitter
from IPython.display import display
from matplotlib import pyplot as plt

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

import pandas as pd
import numpy as np

n_trails = 1
# 读取数据，B超诊断阳性=1，bc_data.csv是要读取的数据。
# data_file = r'D:/20220531-Sychoah/泛癌深度学习/mirna/mirna_norm.csv'
plt.rcParams['figure.dpi'] = 300
# plt.rcParams['figure.figsize'] = (5+n_trails*5, 20)
# plt.rcParams['figure.figsize'] = (5, 15)
event = labels[0]
event_time = labels[1]

def get_ds(s_data, group):
    s_data = s_data[s_data['group'] == group]
    s_data = s_data.dropna(axis=0)
    s_data[event_time] = s_data[event_time] / 12
    s_data[event] = s_data[event].map(lambda x: 1 if x == 'YES' else 0)
    s_data = s_data.drop(['group'], axis=1)
    return s_data

metric = []
row_num = 0

d_train = get_ds(structed_data, 'TCGA-PRAD')
d_test = get_ds(structed_data, 'CPGEA')
display(d_train)
display(d_test)

cph = CoxPHFitter(penalizer=0.024)
# X_train, X_test, y_train, y_test = okcomp.comp1.split_dataset(X_data, y_data, test_size=0.2)
cph.fit(d_train[[c for c in d_train.columns if c not in featrues_not_use]], duration_col=event_time, event_col=event)

for cohort, data_ in  zip(['Train', 'Test'], [d_train, d_test]):
#     y_pred = cph.predict_partial_hazard(data_)
    y_pred = cph.predict_median(data_)
    c_index = cph.score(data_, scoring_method="concordance_index")
    metric.append([cohort, c_index])
    data_ = pd.concat([data_, y_pred], axis=1)
#     display(data_)
    mean = data_.describe()[event_time]['mean']
    data_['HR'] = data_[0.5] < mean
    dem = (data_["HR"] == True)

    results = logrank_test(data_[event_time][dem], data_[event_time][~dem], 
                           event_observed_A=data_[event][dem], event_observed_B=data_[event][~dem])
    p_value = results.p_value
    kmf = KaplanMeierFitter()
    ax = plt.subplot(1, 1, 1)
    plt.title(f"C-index:{c_index:.3f}, p_value={p_value:.5f}")
    if sum(dem):
        kmf.fit(data_[event_time][dem], event_observed=data_[event][dem], label="High Rish")
        kmf.plot_survival_function(ax=ax, color='r')
    if sum(~dem):
        kmf.fit(data_[event_time][~dem], event_observed=data_[event][~dem], label="Low Risk")
        kmf.plot_survival_function(ax=ax, color='g')

    row_num += n_trails
    plt.savefig(f'img/km_{cohort}_clinical.svg', bbox_inches = 'tight')
    plt.show()
    plt.close()
cph.print_summary()
pd.DataFrame(metric, columns=['Train', 'Val'])

In [ ]:
from collections import namedtuple
import onekey_algo.custom.components as okcomp
from onekey_algo import OnekeyDS as okds
from onekey_algo.custom.utils import map2numerical
from onekey_algo.custom.components.comp1 import fillna
import onekey_algo.custom.components as okcomp
from lifelines import CoxPHFitter
from IPython.display import display
from matplotlib import pyplot as plt

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

import pandas as pd
import numpy as np

n_trails = 1
# 读取数据，B超诊断阳性=1，bc_data.csv是要读取的数据。
# data_file = r'D:/20220531-Sychoah/泛癌深度学习/mirna/mirna_norm.csv'
plt.rcParams['figure.dpi'] = 300
# plt.rcParams['figure.figsize'] = (5+n_trails*5, 20)
# plt.rcParams['figure.figsize'] = (5, 15)
event = labels[0]
event_time = labels[1]

def get_ds(s_data, group):
    s_data = s_data[s_data['group'] == group]
    s_data = s_data.dropna(axis=0)
    s_data[event_time] = s_data[event_time] / 12
    s_data[event] = s_data[event].map(lambda x: 1 if x == 'YES' else 0)
    s_data = s_data.drop(['group'], axis=1)
    s_data = s_data.reset_index()
    return s_data

metric = []
row_num = 0

d_train = get_ds(structed_data, 'CPGEA')
d_test = get_ds(structed_data, 'CPGEA')
# subsets = okcomp.comp2.split_dataset(d_train, d_train[event], test_size=0.2, n_trails=5, cv=False, random_state=10)
display(d_train)
display(d_test)

featrues_not_use = ['ID']
for cv, (d_train, d_test) in enumerate(subsets):
    cph = CoxPHFitter(penalizer=0.1)
    # X_train, X_test, y_train, y_test = okcomp.comp1.split_dataset(X_data, y_data, test_size=0.2)
    cph.fit(d_train[[c for c in d_train.columns if c not in featrues_not_use]], duration_col=event_time, event_col=event)

    for cohort, data_ in  zip([f'Train', f'Test'], [d_train, d_test]):
    # for cohort, data_ in  zip(['Train', 'Test'], subsets + [d_test]):
        y_pred = cph.predict_partial_hazard(data_)
#         y_pred = cph.predict_median(data_)
        c_index = cph.score(data_, scoring_method="concordance_index")
        metric.append([f"CV-{cv}", cohort, c_index])
        data_ = pd.concat([data_, y_pred], axis=1)
        mean = data_.describe()[event_time]['mean']
        data_['HR'] = data_[0] > 1
        dem = (data_["HR"] == True)

        results = logrank_test(data_[event_time][dem], data_[event_time][~dem], 
                               event_observed_A=data_[event][dem], event_observed_B=data_[event][~dem])
        p_value = results.p_value
        kmf = KaplanMeierFitter()
        ax = plt.subplot(1, 1, 1)
        plt.title(f"C-index:{c_index:.3f}, p_value={p_value:.5f}")
        if sum(dem):
            kmf.fit(data_[event_time][dem], event_observed=data_[event][dem], label="High Rish")
            kmf.plot_survival_function(ax=ax, color='r')
        if sum(~dem):
            kmf.fit(data_[event_time][~dem], event_observed=data_[event][~dem], label="Low Risk")
            kmf.plot_survival_function(ax=ax, color='g')

        row_num += n_trails
        plt.savefig(f'img/km_{cohort}_pathology.svg', bbox_inches = 'tight')
        plt.show()
        plt.close()
pd.DataFrame(metric, columns=['CV', 'Train', 'Val'])

In [ ]:
cph.print_summary()